# Tutorial 3: Estimating nonlinear diffusion ages

The last tutorial worked on the assumption of linear hillslope diffusion, i.e.
$$ \frac{\partial z}{\partial t} = k\frac{\partial^2z}{\partial x^2}$$.
Some predictions of the linear diffusion equation, like convex hillslope forms, do not hold on steep slopes. This is especially relevant for young terrace risers that start out (more or less) at the angle of repose! Because of this, it is also possible to use a nonlinear diffusion equation in riserfit:
$$ \frac{\partial z}{\partial t} = \frac{\partial}{\partial x}\left(K\frac{\frac{\partial z}{\partial x}}{1-\left(S_c^{-1}\frac{\partial z}{\partial x}\right)^2}\right)$$
There are two caveats: First, we still require the parameters inferred from the linear diffusion fitting, because an initial riser shape is needed for the nonlinear algorithm. Second, the nonlinear diffusion equation does not have an analytical solution and a numerical forward model is needed to find a best-fit profile. The numerical solution is pretty unstable, so the computational cost is greatly increased.

The two functions needed to infer nonlinear diffusion ages and uncertainties are `rf.Riser.compute_best_nonlinear_diffusion_fit()` and `rf.Riser.calculate_nonlin_t_uncertainty()`

In [ ]:
# Some imports
import riserfit as rf
import os
import matplotlib.pyplot as plt
from time import time
import numpy as np

In [ ]:
# load the risers from the last notebook
os.chdir(r"C:\\Users\\Lennart\\lennartGit\\personal\\riserfit\\Tutorials")

# load the two riser instances
terraces = ["T7", "T3"]
fnames = [f"\\Data\\Risers\\Instances\\{t}_Riser_instance.gz" for t in terraces]
risersT7 = rf.load_instance(fnames[0])
risersT3 = rf.load_instance(fnames[1])

# calculate nonlinear diffusion ages
start = time()
risersT7.compute_best_nonlinear_diffusion_fit(
    k=1, # If k = 1, we can interpret the results as diffusion ages (kt)
    S_c=1, # Critical slope
    init_dt=0.25, # Time step size of the forward model
    verbose=True,
    warning_eps=1. # If any slopes are larger than this value, raise a warning. Good indicator of stability
)
risersT7.calculate_nonlin_t_uncertainty(
    k=1,
    S_c=1,
    dt=0.05
)
end = time()
print(f"That took {end - start:.2f} seconds!")

# ... and do the same for T3
start = time()
risersT3.compute_best_nonlinear_diffusion_fit(
    k=1, 
    S_c=1, 
    init_dt=0.25, 
    warning_eps=1.,
    verbose=False
)
risersT3.calculate_nonlin_t_uncertainty(
    k=1,
    S_c=1,
    dt=0.05,
    verbose=False
)
end = time()
print(f"That took {end - start:.2f} seconds!")

In [ ]:
# Let's compare the NONLINEAR diffusion ages of T7 and T3.
bins = np.linspace(0, 10, 10)
fg, ax = plt.subplots(2, 1)
ax[0].hist(
    risersT7.nonlin_best_t,
    bins=bins
);
ax[0].set_title("T7") # this is the young terrace!

ax[1].hist(
    risersT3.nonlin_best_t,
    bins=bins
);
ax[1].set_title("T3") # this one is older!
ax[1].set_xlabel("Nonlinear diffusion age kt [m^2]");

In [ ]:
# And let's look at the dataframe again:
df = risersT7.build_Riser_instance_dataframe()
print(df.head())

In [ ]:
# Save the riser instances after our hard work.
risersT7.save_instance(r"\\Data\\Risers\\Instances\\");
risersT3.save_instance(r"\\Data\\Risers\\Instances\\");